# Prediction Modelling for Hotel Ratings from Reviews, Reviewer Nationality and Hotel Location



## Business Problem

#### Goals
- Determine a relationship between reviews and the rating a reviewer gives.
- Reviewer Nationality can affect ratings

#### Other Impacts on Review Ratings
- Stay Duration
- Reviewer Nationality
- Location of the Hotel
- Number of Reviews
- Tags associated with the Trip

#### Why?
- This could allow the development of review apps and websites that could allow pre-filled rating based on a review.
- Allow improvements of Hotels based on reviews


## Risks and Limitations
- Assumption that Reviews are Honest. 
- Hotel is treated as constant.
- Low Score is solely due factors discussed in written reivew.
- Each review is treated as seperate.
- External Factors are ignored

### Data Access
Data was obtained from [Kaggle](https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe)

## The Review Data

#### Acknowledgements
The data was scraped from Booking.com by [Jason Liu](https://www.kaggle.com/jiashenliu)

#### Data Context
- 515,000 customer reviews
- 1493 luxury hotels within Europe

#### Data Content
- **Hotel_Address**: Address of hotel.
- **Review_Date:** Date when reviewer posted the corresponding review.
- **Average_Score:** Average Score of the hotel, calculated based on the latest comment in the last year.
- **Hotel_Name:** Name of Hotel
- **Reviewer_Nationality:** Nationality of Reviewer
- **Negative_Review:** Negative Review the reviewer gave to the hotel. If the reviewer does not give the negative review, then it should be: 'No Negative'
- **Review_Total_Negative_Word_Counts:** Total number of words in the negative review.
- **Positive_Review:** Positive Review the reviewer gave to the hotel. If the reviewer does not give the negative review, then it should be: 'No Positive'

- **Review_Total_Positive_Word_Counts:** Total number of words in the positive review.
- **Reviewer_Score:** Score the reviewer has given to the hotel, based on his/her experience
- **Total_Number_of_Reviews_Reviewer_Has_Given:** Number of Reviews the reviewers has given in the past.
- **Total_Number_of_Reviews:** Total number of valid reviews the hotel has.
- **Tags:** Tags reviewer gave the hotel.
- **days_since_review:** Duration between the review date and scrape date.
- **Additional_Number_of_Scoring:** There are also some guests who just made a scoring on the service rather than a review. This number indicates how many valid scores without review in there.
- **lat:** Latitude of the hotel
- **lng:** longtitude of the hotel

In [5]:
# Script Name: EDA of Hotel Reviews Data
# Author: Rahul Kumar
# Date: 2-Jan-20
# Description: The purpose is to clean up data in preperation for Model running

import pandas as pd
import numpy as np
from math import sqrt
import seaborn as sns
import scipy as sp
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer

#this supposedly extracts country from a text line
import pycountry

import matplotlib.pyplot as plt
%matplotlib inline

# This actually sets the pandas display to show all rows and columns 
# when you are showing a dataframe, without skipping the center
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
hotels = pd.read_csv('../Hotel_Reviews.csv')

### Preview of the Data
- Note the Review Score is the label to be predicted

In [7]:
hotels.head(4)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968


## Explatory Data Analysis
The Data was in a workable state and not much primary cleaning had to be done.

- Basic Text Cleaning
- Dropped Null Values
- Basic Text and Dataframe Manipulation
- Some relationships were observed

In [8]:
hotels.dtypes

Hotel_Address                                  object
Additional_Number_of_Scoring                    int64
Review_Date                                    object
Average_Score                                 float64
Hotel_Name                                     object
Reviewer_Nationality                           object
Negative_Review                                object
Review_Total_Negative_Word_Counts               int64
Total_Number_of_Reviews                         int64
Positive_Review                                object
Review_Total_Positive_Word_Counts               int64
Total_Number_of_Reviews_Reviewer_Has_Given      int64
Reviewer_Score                                float64
Tags                                           object
days_since_review                              object
lat                                           float64
lng                                           float64
dtype: object

In [9]:
hotels.isnull().sum().to_frame()

,0
Hotel_Address,0
Additional_Number_of_Scoring,0
Review_Date,0
Average_Score,0
Hotel_Name,0
Reviewer_Nationality,0
Negative_Review,0
Review_Total_Negative_Word_Counts,0
Total_Number_of_Reviews,0
Positive_Review,0


In [10]:
hotels.describe()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,lat,lng
count,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,512470.000000,512470.000000
mean,498.081836,8.397487,18.539450,2743.743944,17.776458,7.166001,8.395077,49.442439,2.823803
std,500.538467,0.548048,29.690831,2317.464868,21.804185,11.040228,1.637856,3.466325,4.579425
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.000000,2.500000,41.328376,-0.369758
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.000000,7.500000,48.214662,-0.143372
50%,341.000000,8.400000,9.000000,2134.000000,11.000000,3.000000,8.800000,51.499981,0.010607
75%,660.000000,8.800000,23.000000,3613.000000,22.000000,8.000000,9.600000,51.516288,4.834443
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.000000,10.000000,52.400181,16.429233


In [11]:
hotels.groupby('Hotel_Name').mean()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,lat,lng
Hotel_Name,,,,,,,,,
11 Cadogan Gardens,101.000000,8.700000,15.528302,393.000000,19.974843,7.226415,8.845283,51.493616,-0.159235
1K Hotel,69.000000,7.700000,24.932432,663.000000,15.601351,9.141892,7.861486,48.863932,2.365874
25hours Hotel beim MuseumsQuartier,391.000000,8.800000,16.161103,4324.000000,21.911466,8.722787,8.983309,48.206474,16.354630
41,66.000000,9.600000,8.883495,244.000000,25.300971,6.009709,9.711650,51.498147,-0.143649
45 Park Lane Dorchester Collection,27.000000,9.400000,6.750000,68.000000,11.535714,7.214286,9.603571,51.506371,-0.151536
88 Studios,197.000000,8.400000,23.936819,955.000000,21.464052,7.427015,8.489107,51.499279,-0.209073
9Hotel Republique,100.000000,8.800000,16.950820,857.000000,19.338798,8.633880,8.743716,48.870842,2.360586
A La Villa Madame,24.000000,8.800000,8.463415,185.000000,19.634146,5.829268,8.853659,48.848861,2.331526
ABaC Restaurant Hotel Barcelona GL Monumento,10.000000,8.800000,35.225806,111.000000,18.677419,10.258065,8.464516,41.410694,2.136294
